# Feature Enginnering Part 2

Here, the Feature Engineering process, defined on Part 1, is applied to all data.

## Libraries

In [1]:
import numpy as np 
import pandas as pd 
import tsfresh
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute
from cnr_methods import get_simplified_data, transform_data

## Feature Engineering Functions

Here, all the process created on Part 1 is organized on Functions before applying to the data.

In [2]:
def get_selected_features(n_features):
    selected_features = pd.read_csv(r'Feature Selection\Importance_WF1.csv')
    selected_features = selected_features[:n_features]['feature']
    return selected_features

In [3]:
def get_manual_features(feature_data):

    index = feature_data.index
    features = ['T', 'CLCT', 'U_100m','V_100m','U_10m','V_10m']

    # Wind Speed Vector
    feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
    feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
    feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
    feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])

    feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
    feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

    # Time Relative Variables 

    for column in features:
        feature_data[column + '_last_week'] = feature_data[column].shift(7) # Values for Last Week
        feature_data[column + '_last_month'] = feature_data[column].shift(30) # Values for Last Month

    feature_data['Month_Number'] = feature_data.index.month # Month Number
    feature_data['Quarter_Number'] = feature_data.index.quarter # Quarter Number

    mean_month = feature_data.groupby('Month_Number').mean()[features] # Month Mean
    median_month = feature_data.groupby('Month_Number').median()[features] # Month Median
    variance_month = feature_data.groupby('Month_Number').var()[features] # Month Variance

    mean_quarter = feature_data.groupby('Quarter_Number').mean()[features] # Quarter Mean
    median_quarter = feature_data.groupby('Quarter_Number').median()[features] # Quarter Median
    variance_quarter = feature_data.groupby('Quarter_Number').var()[features] # Quarter Variance

    mean_month.columns = mean_month.columns + '_Month_Mean'
    median_month.columns = median_month.columns + '_Month_Median'
    variance_month.columns = variance_month.columns + '_Month_Variance'
    mean_quarter.columns = mean_quarter.columns + '_Quarter_Mean'
    median_quarter.columns = median_quarter.columns + '_Quarterh_Median'
    variance_quarter.columns = variance_quarter.columns + '_Quarter_Variance'

    feature_data = feature_data.merge(mean_month,on='Month_Number',how='left')
    feature_data = feature_data.merge(median_month,on='Month_Number',how='left')
    feature_data = feature_data.merge(variance_month,on='Month_Number',how='left')
    feature_data = feature_data.merge(mean_quarter,on='Quarter_Number',how='left')
    feature_data = feature_data.merge(median_quarter,on='Quarter_Number',how='left')
    feature_data = feature_data.merge(variance_quarter,on='Quarter_Number',how='left')
    feature_data.index = index

    # Periodical Features

    day = feature_data.index.day
    hour = feature_data.index.hour
    minute = feature_data.index.minute
    dayofweek = feature_data.index.dayofweek
    dayofyear = feature_data.index.dayofyear
    days_in_month = feature_data.index.days_in_month

    feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

    feature_data["cos_hour"], feature_data["sin_hour"] = (
        np.cos(2 * np.pi * hour / 24),
        np.sin(2 * np.pi * hour / 24),
        )

    feature_data["cos_minute"], feature_data["sin_minute"] = (
        np.cos(2 * np.pi * minute / 60),
        np.sin(2 * np.pi * minute / 60),
    )

    feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
        np.cos(2 * np.pi * (dayofyear - 1) / 365),
        np.sin(2 * np.pi * (dayofyear - 1) / 365),
    )

    feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
        np.cos(2 * np.pi * dayofweek / 7),
        np.sin(2 * np.pi * dayofweek / 7),
    )

    # Distance from Max and Min

    for column in features:
        feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
        feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
        feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
        feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

    # Dropping Base Features 
    #features.append(['Month_Number','Quarter Number'])
    feature_data = feature_data.drop(features,axis=1)

    return feature_data

In [4]:
def get_tsfresh_features(data):
    tsfresh_data = pd.DataFrame()
    for variable in ['U_100m','V_100m','U_10m','V_10m','T','CLCT']: 
        df_shift, y = make_forecasting_frame(data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
        X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,show_warnings=False,n_jobs=3)
        X['Feature'] = variable
        tsfresh_data = tsfresh_data.append(X)

    # Formmating the Data
    tsfresh_data = tsfresh_data.pivot(columns='Feature')
    tsfresh_data.columns = tsfresh_data.columns.map('{0[0]}|{0[1]}'.format)
    #tsfresh_data = tsfresh_data.loc[:, tsfresh_data.apply(pd.Series.nunique) != 1] # Remove Constant Columns

    return tsfresh_data

In [5]:
def get_features_data(X): 
    feature_data = pd.DataFrame()
    for WF in X['WF'].unique():
        X_WF = X[X['WF']==WF]

        X_manual = get_manual_features(X_WF)
        X_tsfresh = get_tsfresh_features(X_WF)
        X_WF = pd.concat([X_manual,X_tsfresh],axis=1)
        X_WF['WF'] = WF
        feature_data = pd.concat([feature_data,X_WF],axis=0)

    feature_data = pd.concat([X,feature_data],axis=1) 

    return feature_data

## Process Application

Here, the Feature Engineering is properly applied on the Full Data.

In [6]:
X,y_train = get_simplified_data()

X_train = X[X['Set']=='Train']
X_test = X[X['Set']=='Test']

In [7]:
X_train = get_features_data(X_train)

Feature Extraction: 100%|██████████| 15/15 [01:46<00:00,  7.13s/it]


In [8]:
X_test = get_features_data(X_test)

Feature Extraction: 100%|██████████| 15/15 [01:33<00:00,  6.22s/it]


In [23]:
X_train['Set'] = 'Train'
X_test['Set'] = 'Test'

feature_data = pd.concat([X_train,X_test],axis=0)

feature_data = feature_data.loc[:,~feature_data.columns.duplicated()]


feature_data.to_csv('Feature Selection/Selected_Features_Data.csv')

In [24]:
feature_data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set,Wind Speed 100m,...,value__variance|U_100m,value__variance|U_10m,value__variance|V_100m,value__variance|V_10m,value__variance_larger_than_standard_deviation|CLCT,value__variance_larger_than_standard_deviation|T,value__variance_larger_than_standard_deviation|U_100m,value__variance_larger_than_standard_deviation|U_10m,value__variance_larger_than_standard_deviation|V_100m,value__variance_larger_than_standard_deviation|V_10m
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train,3.958410,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train,2.831607,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 03:00:00,3,WF1,-1.707402,-0.853745,0.997093,-1.415138,287.000000,98.367235,Train,1.908954,...,0.008649,0.382112,0.820564,0.003824,1.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train,7.238384,...,0.095117,0.425138,1.045847,0.253942,1.0,0.0,0.0,0.0,1.0,0.0
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train,6.647232,...,6.458769,0.467797,4.356622,0.202684,1.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-30 18:00:00,73900,WF6,-3.039992,-4.250952,-0.530133,-1.421466,292.389997,0.000000,Test,5.226102,...,1.571467,0.519446,19.502184,6.778447,1.0,1.0,1.0,0.0,1.0,1.0
2019-09-30 19:00:00,73901,WF6,-3.204700,-4.348800,0.121283,-0.885161,292.480000,0.000000,Test,5.402052,...,1.917278,0.519429,13.911188,4.784000,1.0,1.0,1.0,0.0,1.0,1.0
2019-09-30 20:00:00,73902,WF6,-0.746600,-4.334300,0.080257,-0.690724,291.340000,0.000000,Test,4.398132,...,2.271335,0.535901,8.168720,3.101331,1.0,1.0,1.0,0.0,1.0,1.0
2019-09-30 21:00:00,73903,WF6,-0.169990,-2.814300,-0.057787,-0.809309,289.635009,0.000000,Test,2.819429,...,2.123036,0.548414,4.847935,2.601988,1.0,1.0,1.0,0.0,1.0,1.0
